# Motif Enrichment
- The motif hits matrix save the max loglikelyhood score of a motif in that DMR
- Before run the motif enrichment, I filter the scores for each motif using cutoff defined by
    - `np.quantile(motif_score[motif_score > 0]) * mask_quantile_to99`
- This filter will remove hits with low score, which is likely to be false positive
- Its likely boosting the significance of real enrichment

In [1]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pybedtools
import scipy.stats as stats
import seaborn as sns
from scipy.sparse import csr_matrix
from statsmodels.stats.multitest import multipletests
import pathlib

In [2]:
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = 'IT-L23+ACA'

motif_hits_path = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifScan.h5ad'
dmr_hits_path = f'/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/HypoDMR/{group}.DMS1.bed'
output_dir = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifEnrichment'


In [3]:
# Parameters
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = "group0"
motif_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/snm3C-DMR/MotifScan.h5ad"
dmr_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/snm3C-DMR/HypoDMR/group0.DMS1.bed"
output_dir = "/home/hanliu/project/mouse_rostral_brain/DMR/DGmCHGroup/snm3C-DMR/MotifEnrichment"


In [4]:
output_dir = pathlib.Path(output_dir)
output_dir.mkdir(exist_ok=True)

## DMR hits

In [5]:
dmr_bed_df = pd.read_csv(dmr_hits_path, sep='\t', header=None, index_col=3)
dmr_bed_df.head()

,0,1,2
3,,,
DGm3c_11,chr1,4731880,4731880
DGm3c_12,chr1,4767514,4767514
DGm3c_20,chr1,5750645,5750645
DGm3c_24,chr1,6487486,6487486
DGm3c_27,chr1,6585232,6585232


In [6]:
motif_hits = anndata.read_h5ad(motif_hits_path)
motif_hits = motif_hits[dmr_bed_df.index, :].copy()
motif_ids = motif_hits.var_names
motif_hits.X = motif_hits.X.todense()
motif_hits

AnnData object with n_obs × n_vars = 21436 × 719 
    obs: 'chrom', 'start', 'end'

In [7]:
# mask small motif scores
motif_cutoff = pd.Series(np.apply_along_axis(lambda i: np.quantile(i[i>0], 0.99) * mask_quantile_to99, 
                                             0, motif_hits.X),
                         index=motif_hits.var_names)
# only keep value larger than the cutoff for each motif
motif_hits.X = np.multiply(motif_hits.X, (motif_hits.X > motif_cutoff.values[None, :]))

## Background Hits

In [8]:
background_motif_hits = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/BackgroundDMR/MotifScan.h5ad'
)

In [9]:
dmr_bed = pybedtools.BedTool().from_dataframe(dmr_bed_df)
bg_bed = pybedtools.BedTool().from_dataframe(
    background_motif_hits.obs.reset_index().iloc[:, [1, 2, 3, 0]])

In [10]:
# exclude background that overlap with DMR
bg_no_overlap = bg_bed.intersect(dmr_bed, v=True)
use_bg = bg_no_overlap.to_dataframe().iloc[:, -1].values
background_motif_hits = background_motif_hits[use_bg, :]

# make sure col in same order
background_motif_hits = background_motif_hits[:, motif_ids].copy()

In [11]:
judge_matrix = csr_matrix(background_motif_hits.X > motif_cutoff.values[None, :])
background_motif_hits.X = background_motif_hits.X.multiply(judge_matrix)

In [12]:
background_motif_hits

AnnData object with n_obs × n_vars = 338413 × 719 
    obs: 'chrom', 'start', 'end'

## Motif hits contingency table

In [13]:
# calculate motif occurence, not considering hits here
pos = (motif_hits.X > 0).sum(axis=0)
pos_total = motif_hits.shape[0]

neg = (background_motif_hits.X > 0).sum(axis=0)
neg_total = background_motif_hits.shape[0]

In [14]:
tables = {}
for motif, _pos, _neg in zip(motif_ids, pos.A1, neg.A1):
    table = [[_pos, pos_total - _pos], [_neg, neg_total - _neg]]
    tables[motif] = table

In [15]:
results = {}
with ProcessPoolExecutor(20) as executor:
    fs = {}
    for motif, t in tables.items():
        f = executor.submit(stats.fisher_exact, t, alternative='greater')
        fs[f] = motif

    for f in as_completed(fs):
        motif = fs[f]
        odds, p = f.result()
        results[motif] = {'oddsratio': odds, 'p_value': p}
motif_enrich_df = pd.DataFrame(results).T

_, p, _, _ = multipletests(motif_enrich_df['p_value'], method='fdr_bh')
motif_enrich_df['adj_p'] = p

motif_enrich_df['-lgp'] = -np.log10(motif_enrich_df['adj_p']).replace(
    -np.inf, -300)

records = {}
for motif, t in tables.items():
    tp, tn = t[0]
    fp, fn = t[1]
    tp_rate = tp / pos_total
    fp_rate = fp / neg_total
    records[motif] = dict(tp=tp,
                          tn=tn,
                          fp=fp,
                          fn=fn,
                          tp_rate=tp_rate,
                          fp_rate=fp_rate)
counts = pd.DataFrame(records).T
motif_enrich_df = pd.concat([motif_enrich_df, counts], axis=1, sort=True)

In [16]:
motif_enrich_df['Group'] = group

In [17]:
# final filter
filtered_motif_df = motif_enrich_df[(motif_enrich_df['oddsratio'] > or_cutoff)
                                    &
                                    (motif_enrich_df['-lgp'] > neg_lgp_cutoff)]
filtered_motif_df.shape[0]

23

In [18]:
motif_enrich_df.to_msgpack(output_dir / f'{group}.motif_enrichment.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.


In [19]:
pybedtools.cleanup()

In [20]:
filtered_motif_df

,oddsratio,p_value,adj_p,-lgp,tp,tn,fp,fn,tp_rate,fp_rate,Group
MA0027.2,1.532039,7.197724e-43,5.750182e-41,40.240318,1330.0,20106.0,14007.0,324406.0,0.062045,0.041390,group0
MA0068.2,1.849589,1.957932e-19,3.199439e-18,17.494926,281.0,21155.0,2413.0,336000.0,0.013109,0.007130,group0
MA0630.1,1.616279,7.236935e-61,8.672260e-59,58.061868,1550.0,19886.0,15569.0,322844.0,0.072308,0.046006,group0
MA0644.1,1.582287,7.485086e-20,1.312628e-18,17.881858,494.0,20942.0,4971.0,333442.0,0.023045,0.014689,group0
MA0654.1,1.616279,7.236935e-61,8.672260e-59,58.061868,1550.0,19886.0,15569.0,322844.0,0.072308,0.046006,group0
MA0662.1,1.558810,6.078954e-53,6.243954e-51,50.204540,1552.0,19884.0,16137.0,322276.0,0.072402,0.047684,group0
MA0699.1,1.641077,5.147633e-31,1.480459e-29,28.829604,700.0,20736.0,6821.0,331592.0,0.032655,0.020156,group0
MA0701.2,1.616279,7.236935e-61,8.672260e-59,58.061868,1550.0,19886.0,15569.0,322844.0,0.072308,0.046006,group0
MA0710.1,1.811691,2.601661e-25,6.450324e-24,23.190418,400.0,21036.0,3515.0,334898.0,0.018660,0.010387,group0
MA0716.1,1.616279,7.236935e-61,8.672260e-59,58.061868,1550.0,19886.0,15569.0,322844.0,0.072308,0.046006,group0
